In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import MaxPool2D, Conv2DTranspose, Concatenate, Input

In [ ]:
!pip install opencv-python

### Load images

In [2]:
import os
import cv2

folder_path = "C:/Users/DELL/Desktop/archive/images"
# Get list of all image files in folder
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".png")]

In [3]:
img_size = (256,256)

In [4]:
# Load images
images = []
for image_file in image_files:
    img = cv2.imread(image_file)
    img = cv2.resize(img, img_size)
    img = img / 255.0
    img = img.astype(np.float32)
    images.append(img)

### Load masks

In [5]:
folder_path = "C:/Users/DELL/Desktop/archive/masks"
# Get list of all image files in folder
mask_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".png")]

In [6]:
# Load masks
masks = []
for mask_file in mask_files:
    mask = cv2.imread(mask_file,cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, img_size)
    mask = mask / 255.0
    mask = mask.astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    masks.append(mask)

### Display images

In [7]:
# Display first mask
cv2.imshow("First image", images[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
# Display first mask
cv2.imshow("First mask", masks[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
height, width, channels = images[0].shape
print(f"Image size: {width}x{height}", " channels : ", channels)

Image size: 256x256  channels :  3


In [10]:
height, width, channels = masks[0].shape
print(f"Image size: {width}x{height}", " channels : ", channels)

Image size: 256x256  channels :  1


### Conv Block

In [11]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

### Encoder Block

In [12]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

### Decoder Block

In [13]:
def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

### Build U-Net

In [14]:
def unet(input_shape):
    
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

model = unet((256,256,3))
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

 rmalization)                   )                                                                 
                                                                                                  
 activation_8 (Activation)      (None, 16, 16, 1024  0           ['batch_normalization_8[0][0]']  
                                )                                                                 
                                                                                                  
 conv2d_9 (Conv2D)              (None, 16, 16, 1024  9438208     ['activation_8[0][0]']           
                                )                                                                 
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 16, 16, 1024  4096       ['conv2d_9[0][0]']               
 rmalization)                   )                                                                 
          

                                                                                                  
 conv2d_16 (Conv2D)             (None, 256, 256, 64  73792       ['concatenate_3[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_16 (BatchN  (None, 256, 256, 64  256        ['conv2d_16[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 activation_16 (Activation)     (None, 256, 256, 64  0           ['batch_normalization_16[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_17

### Split the data

random_state parameter to ensure that the same random seed is used for both splits.

In [15]:
split = 0.2 
split_size = int(len(images) * split)

In [17]:
from sklearn.model_selection import train_test_split

# Split the images and masks sets into train and test data
train_i, test_i = train_test_split(images, test_size=split_size, random_state=42) 
train_m, test_m = train_test_split(masks, test_size=split_size, random_state=42) 

# Split the train set into train and val sets
train_i,val_i = train_test_split(train_i, test_size=split_size, random_state=42)
train_m,val_m = train_test_split(train_m, test_size=split_size, random_state=42)


In [18]:
print(len(train_i),"  ",len(val_i),"  ",len(test_i))
print(len(train_m),"  ",len(val_m),"  ",len(test_m))

1840    612    612
1840    612    612


In [28]:
batch_size = 16
lr = 1e-4
num_epochs = 50

In [20]:
def tf_dataset(X, Y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

In [21]:
train_dataset = tf_dataset(train_i, train_m, batch=batch_size)
valid_dataset = tf_dataset(val_i, val_m, batch=batch_size) 

In [22]:
train_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None))>

In [23]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [24]:
model = unet((256, 256, 3))
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef,'accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

history = model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/50
115/115 [==============================] - 6128s 53s/step - loss: 0.8627 - dice_coef: 0.1373 - accuracy: 0.8469 - val_loss: 0.9790 - val_dice_coef: 0.0210 - val_accuracy: 0.8628
Epoch 2/50
  9/115 [=>............................] - ETA: 1:09:49 - loss: 0.7720 - dice_coef: 0.2280 - accuracy: 0.9590